In [229]:
from datasets import load_from_disk
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
import torch
from tqdm.auto import tqdm
import evaluate
import random
import argparse
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk import pos_tag
import string

In [203]:
def custom_transform(example):
    
    ################################
    ##### YOUR CODE BEGINGS HERE ###
    
    # Design and implement the transformation as mentioned in pdf
    # You are free to implement any transformation but the comments at the top roughly describe
    # how you could implement two of them --- synonym replacement and typos.
    
    # You should update example["text"] using your transformation
        
    raise NotImplementedError
    
    
    ##### YOUR CODE ENDS HERE ######
    
    return example




In [204]:
# Define the directory where the dataset is saved
load_directory = "dataset/"

# Load the dataset from the specified directory
loaded_data = load_from_disk(load_directory)

# Now you can access the individual splits (train, test, unsupervised) as follows:
train_dataset = loaded_data["train"]
test_dataset = loaded_data["test"]
unsupervised_dataset = loaded_data["unsupervised"]

In [205]:
# replacement probability
probability = 0.6

# Generate a random number between 0 and 1
random_number = random.random()

In [281]:
example = test_dataset[15]['text']

In [282]:
example

'Now, I LOVE Italian horror films. The cheesier they are, the better. However, this is not cheesy Italian. This is week-old spaghetti sauce with rotting meatballs. It is amateur hour on every level. There is no suspense, no horror, with just a few drops of blood scattered around to remind you that you are in fact watching a horror film. The "special effects" consist of the lights changing to red whenever the ghost (or whatever it was supposed to be) is around, and a string pulling bed sheets up and down. Oooh, can you feel the chills? The DVD quality is that of a VHS transfer (which actually helps the film more than hurts it). The dubbing is below even the lowest "bad Italian movie" standards and I gave it one star just because the dialogue is so hilarious! And what do we discover when she finally DOES look in the attic (in a scene that is daytime one minute and night the next)...well, I won\'t spoil it for anyone who really wants to see, but let\'s just say that it isn\'t very "novel"

# Synonym

In [207]:
def get_wordnet_pos(treebank_tag): # Convert POS-Tag from treebank_tag to Wordnet tags
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return None  # If no match is found, return None
    


In [208]:
def replace_with_synonym(text: str, pct_synonyms: float) -> str:
    """Replace {pct_synonyms}% of words in a sentence with their synonyms
    :param text (str):
    :param pct_synonyms (float): percentage of words to be replaced. Between 0 and 1
    :return: transformed text
    """
    # Tokenize sentence
    word_list  = word_tokenize(text)
    n_tokens = len(word_list)
    n_synonyms = int(n_tokens*pct_synonyms)

    # Perform POS tagging to help selecting synonym
    pos_tags = pos_tag(word_list)
    syn_dict = {}
    for i in range(n_synonyms):
        chosen_word = random.choice(word_list)
        synonyms = wn.synsets(chosen_word)

        # If chosen word doesn't have a synonym
        # retry until finding a word that does.
        max_retry = len(word_list)
        retry = 0 
        while (len(synonyms) == 0 and retry <= max_retry):
            chosen_word = random.choice(word_list)
            synonyms = wn.synsets(chosen_word)

            if len(synonyms) > 0:
                # Print the word and its POS tag
                chosen_word_tag = [tag for word, tag in pos_tags if word == chosen_word][0]
                chosen_word_wordnet_tag = get_wordnet_pos(chosen_word_tag)
                
                selected_synonym = chosen_word
                for syn in synonyms:
                    # Only consider synonyms that have the same POS_TAG
                    if syn.pos() == chosen_word_wordnet_tag:
                        for lemma in syn.lemmas():
                            if (lemma.name() != chosen_word):
                                # Select the first synonym that appears
                                selected_synonym = lemma.name().replace('_', ' ') # Replace underscores for multi-word synonyms
                                break
                        break
                    break
                if selected_synonym == chosen_word:
                    synonyms = [] # continue while loop
                else:
                    # print("selected_synonym: ", selected_synonym)
                    syn_dict[chosen_word] = selected_synonym

    # return syn_dict
    # Replace selected words
    for old, new in syn_dict.items():
        text = text.replace(old, new)
    return text, syn_dict

In [209]:
N = 5
for i, data in enumerate(test_dataset):
    sentence = data["text"]
    print("Original\n", sentence)
    sentence_new, syn_dict = replace_with_synonym(sentence, 0.40)
    print("Synonym\n", sentence_new)
    print(syn_dict)
    if i > N:
        break

Original
 I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they hav

# Typos

In [210]:
example = test_dataset[0]['text']

In [211]:
word_list = word_tokenize(example)
word = word_list[5]

In [212]:
word

'willing'

In [213]:
keyboard_layout = {
    'Q': ['W', 'A', 'S'], 'W': ['Q', 'E', 'A', 'S', 'D'], 'E': ['W', 'R', 'S', 'D', 'F'], 'R': ['E', 'T', 'D', 'F', 'G'], 'T': ['R', 'Y', 'F', 'G', 'H'], 
    'Y': ['T', 'U', 'G', 'H', 'J'], 'U': ['Y', 'I', 'H', 'J', 'K'], 'I': ['U', 'O', 'J', 'K', 'L'], 'O': ['I', 'P', 'K', 'L'], 'P': ['O', 'L'],
    'A': ['Q', 'W', 'S', 'Z', 'X'], 'S': ['Q', 'W', 'E', 'A', 'D', 'Z', 'X', 'C'], 'D': ['W', 'E', 'R', 'S', 'F', 'X', 'C', 'V'], 
    'F': ['E', 'R', 'T', 'D', 'G', 'C', 'V', 'B'], 'G': ['R', 'T', 'Y', 'F', 'H', 'V', 'B', 'N'], 'H': ['T', 'Y', 'U', 'G', 'J', 'B', 'N', 'M'], 
    'J': ['Y', 'U', 'I', 'H', 'K', 'N', 'M'], 'K': ['U', 'I', 'O', 'J', 'L', 'M'], 'L': ['I', 'O', 'P', 'K'], 'Z': ['A', 'S', 'X'], 'X': ['Z', 'A', 'S', 'D', 'C'],
    'C': ['X', 'S', 'D', 'F', 'V'], 'V': ['C', 'D', 'F', 'G', 'B'], 'B': ['V', 'F', 'G', 'H', 'N'], 'N': ['B', 'G', 'H', 'J', 'M'], 'M': ['N', 'H', 'J', 'K'],
    'I': ['U', 'O', 'J', 'K', 'L'], 'W': ['Q', 'E', 'A', 'S', 'D']
}

In [214]:
index = random.randint(0, len(word) - 2)

In [245]:
def typo_qwert(word: str, index: int):
    """Replace words close to each other in the QWERT keyboard
    :param word: 
    :param index: index of caracter to be replaced
    """
    # 
    char = word[index]
    if char.upper() in keyboard_layout.keys():
        random_typo = random.choice(keyboard_layout[char.upper()]).lower()
        word = word[:index] + random_typo + word[index+1:]

    return word

In [246]:
def typo_swap_characeters(word: str, index: int):
    """Swap character at index with index + 1
    :param word: 
    :param index: index of caracter to be replaced
    """
    char_list = list(word)
    next_index = index + 1
    char_list[index], char_list[next_index] = char_list[next_index], char_list[index]
    word_swapped = "".join(char_list)
    return word_swapped
    

In [247]:
def typo_remove_character(word: str, index: int):
    """Delete a character
    :param word: 
    :param index: index of caracter to be replaced
    """
    word = word[:index] + word[index + 1:]
    return word

In [248]:
def typo_add_character(word: str, index: int):
    """Add character 
    :param word: 
    :param index: index of caracter to be replaced
    """
    random_char = random.choice(string.ascii_letters)

    return word[:index] + random_char + word[index:]

In [274]:
def add_typos(text: str, pct_typos: float, typos_list=['swap', 'remove', 'add', 'qwert']) -> str:
    """Replace {pct_typos}% of words in a sentence with their syntetic generated typo
    :param text (str):
    :param pct_typos (float): percentage of words to be modified. Between 0 and 1
    :return: transformed text
    """
    # Tokenize sentence
    word_list  = word_tokenize(text)


    # word_list words that have at least 3 characters
    word_list_indices_filtered = [i for i in range(len(word_list)) if len(word_list[i]) >= 3]

    n_tokens = len(word_list_indices_filtered) # only count the tokens that have >= characters
    n_typos = int(n_tokens*pct_typos) # number of typos

    # Typos functions
    typos_functions = {
        'swap': typo_swap_characeters,
        'remove': typo_remove_character,
        'add': typo_add_character,
        'qwert': typo_qwert
    }

    # Select indices based on pct_typos
    word_indices = [index for index in range(len(word_list)) if index in word_list_indices_filtered]
    selected_indices = random.sample(word_indices, k = n_typos)

    # Transformed words
    transformed_words = {}
    for index in selected_indices:

        word = word_list[index]
        typo_index= random.randint(0, len(word) - 2)
        # Randomly select typo technique
        selected_typo = random.choice(typos_list)

        # Apply typo function
        word_transformed = typos_functions[selected_typo](word, typo_index)
        transformed_words[index] = word_transformed

    
    # Replace orginal words with sythetic generated typos
    for i, new_word in transformed_words.items():
        word_list[i] = new_word

    # Put sentence together again:
    text = TreebankWordDetokenizer().detokenize(word_list)
    
    
    return text

In [275]:
example

'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they have

In [276]:
add_typos(example, 0.6)

'I qlove sci-fi and am willing to pMut up wjth a lot . Si-fi movies/TV xre usualy underfunded, under-appreciated and misuderstood . I tridd to liDke ths, I really dd, but it is to good TV sci-fi as BPabylon 5 is to tar Tek (the Soriginal). SZilly prostheMtics, ceap cardboMard sets, stilted dialogues, CG tyat does nu\'t mach the background, and painfulpy one-dimensional chaarcters cNan not be overcoem wiBth a \'ssci-fi\' detting . (I\'m sjre thee are thlse of you tout there wtho tihnk Babyon 5 is good wci-fi TV . It\'s nlt . It\'s cliczhéd aNnd uninspiring .) Whlie US vewers Bmight like emotin and character developmet, sci-fi is a genre that does nit take tiself seriousyl (cf . Star rek). It may treat important issuse, yt not as a esrious philosophy . It\'s really difficult to crae about hte characters hree as twhey ae not sinmply foolish, just jissing a sprak of Ulife . Teir ations adn reactjons rae wooden and predicatble, often painful to watch . hTe makmers of Ebarth KgOW it\'s rubib

# Expand Contractions

# Testing

In [65]:
word_list  = word_tokenize(example)

In [67]:
chosen_word = random.choice(word_list)
synonyms = wn.synsets(chosen_word)

# If chosen word doesn't have a synonym select retry until finding a word that does.
max_retry = len(word_list)
retry = 0 
while (len(synonyms) == 0 and retry <= max_retry):
    chosen_word = random.choice(word_list)
    synonyms = wn.synsets(chosen_word)

In [74]:
# Perform POS tagging
pos_tags = pos_tag(word_list)

# Print the word and its POS tag
chosen_word_tag = [tag for word, tag in pos_tags if word == chosen_word][0]

In [79]:
synonym_list = set()
for syn in synonyms:
    # Only consider synonyms that have the same POS_TAG
    if syn.pos() == chosen_word_wordnet_tag:
        for lemma in syn.lemmas():
            if (lemma.name() != chosen_word) & (lemma):

                synonym_list.add(lemma.name().replace('_', ' ')) # Replace underscores for multi-word synonyms